In [8]:
from pymongo import MongoClient
import pandas as pd
import folium

In [9]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


In [10]:
db, coll = connectCollection('companies','design')

In [44]:
def getNearby(lat,lng,comptype,radius=10000):
    db, coll = connectCollection('companies',comptype)
    results = list(coll.find(
        {"coords":
         {"$near":
          {"$geometry":
           {
               "type":"Point",
               "coordinates":[lng,lat]
           },
           "$maxDistance":radius
          }
         }
        }
    ))
    total = len(results)
    print("Total", total)
    return results if total > 0 else None

getNearby(40.4381307,-3.8199663,"airports")

Total 2


[{'_id': ObjectId('5dd2fe1ecdd0dd41e88a11b0'),
  'airport': 'Cuatro Vientos Airport',
  'coords': {'coordinates': [-3.7851400375, 40.3707008362], 'type': 'Point'},
  'country': 'Spain',
  'latitude': 40.3707008362,
  'longitude': -3.7851400375},
 {'_id': ObjectId('5dd2fe1ccdd0dd41e889edae'),
  'airport': 'Cuatro Vientos',
  'coords': {'coordinates': [-3.785144, 40.370678], 'type': 'Point'},
  'country': 'Spain',
  'latitude': 40.370678,
  'longitude': -3.785144}]

In [13]:
print(len(design_paloalto))

56


In [14]:
map_city=folium.Map(location=[37.4374154,-122.1722525], zoom_start=12)
for c in old_comp_paloalto:
    folium.Marker(c['office_coords']['coordinates'][::-1],
                    radius=2,
                   
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)


In [15]:
map_city

In [ ]:
#empezar buscando la distancia entre dos puntos.

In [16]:
design_paloalto[0]

{'_id': ObjectId('5dd42ff635e2f878c2104b35'),
 'category_code': 'mobile',
 'founded_year': 2005.0,
 'name': 'Mozes',
 'office_coords': {'coordinates': [-122.166911, 37.445667], 'type': 'Point'}}

In [29]:
prueba=coll.find( { 'office_coords': { "$geoWithin": { "$centerSphere": [ [ -122.166911 ] ,
                                                     100 / 3963.2 ] } } } )

In [30]:
list(prueba)

OperationFailure: database error: Can't canonicalize query: BadValue bad geo query: { $geoWithin: { $centerSphere: [ [ -122.166911 ], 0.02523213564796124 ] } }

In [ ]:
var neighborhood = db.neighborhoods.findOne( { geometry: { $geoIntersects: { $geometry: { type: "Point", coordinates: [ -73.93414657, 40.82302903 ] } } } } )
db.restaurants.find( { location: { $geoWithin: { $geometry: neighborhood.geometry } } } ).count()